---
title: "Assignment 1 - Wendy's (WEN) Stock Data Analysis"
author: "Maxim"
format: pdf
---

\newpage
\tableofcontents
\listoffigures
\listoftables
\newpage

## Exercise Warm Up
Some explanation here.


In [11]:
2+2

[1] 4